# Исследования результатов опроса по определнию лояльности клиентов

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Знакомство с данными</a></span><ul class="toc-item"><li><span><a href="#Таблица-user" data-toc-modified-id="Таблица-user-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Таблица user</a></span></li><li><span><a href="#Таблица-location" data-toc-modified-id="Таблица-location-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Таблица location</a></span></li><li><span><a href="#Таблица-age_segment" data-toc-modified-id="Таблица-age_segment-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Таблица age_segment</a></span></li><li><span><a href="#Таблица-traffic_segment" data-toc-modified-id="Таблица-traffic_segment-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Таблица traffic_segment</a></span></li><li><span><a href="#Таблица-lifetime_segment" data-toc-modified-id="Таблица-lifetime_segment-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Таблица lifetime_segment</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Формирование-SQL-запроса" data-toc-modified-id="Формирование-SQL-запроса-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Формирование SQL запроса</a></span></li><li><span><a href="#Дэшборд-Tablaeu" data-toc-modified-id="Дэшборд-Tablaeu-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Дэшборд Tablaeu</a></span></li><li><span><a href="#Презентация" data-toc-modified-id="Презентация-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Презентация</a></span></li></ul></div>

## Знакомство с данными

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

Данные находятся в SQL базе `telecomm_csi.db`. Откроем ее и посмотрим на таблицы

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

### Таблица user

Сначала посмотрим на таблицу `user`

In [3]:
user = pd.read_sql('SELECT * FROM user;', engine)
user.head(10)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,34,4,6,8,6
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,55,5,7,8,10
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,28,4,14,5,10
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,41,4,5,8,10
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,9,5,8,8,9


Это основная таблица, с которой мы будем работать. Она содержит информацию о показателе `nps_score` который нам необходимо исследовать, остальные таблицы дают нам информацию о расшифровке столбцов с `_id` в названии и с помощью запроса будут присоединены к основной.

Посмотрим обшую информацию

Техническим задание требуется расшифровать заначения столбца `gender_segment` для лучшей читабельности. В описании к базе данных сказано, что 1 означает "женщина", а 2 "мужчина". Проверим, что в столбце нет лишних значений

In [4]:
user.info()
user.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         502493 non-null  object 
 1   lt_day          502493 non-null  int64  
 2   age             501939 non-null  float64
 3   gender_segment  501192 non-null  float64
 4   os_name         502493 non-null  object 
 5   cpe_type_name   502493 non-null  object 
 6   location_id     502493 non-null  int64  
 7   age_gr_id       502493 non-null  int64  
 8   tr_gr_id        502493 non-null  int64  
 9   lt_gr_id        502493 non-null  int64  
 10  nps_score       502493 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 42.2+ MB


,count,mean,std,min,25%,50%,75%,max
lt_day,502493.0,1868.841439,1683.701762,-21.0,533.0,1239.0,3064.0,9162.0
age,501939.0,39.621946,11.188249,10.0,31.0,38.0,47.0,89.0
gender_segment,501192.0,0.543588,0.498097,0.0,0.0,1.0,1.0,1.0
location_id,502493.0,31.237321,15.175477,1.0,20.0,30.0,41.0,62.0
age_gr_id,502493.0,4.011045,1.159203,1.0,3.0,4.0,5.0,8.0
tr_gr_id,502493.0,8.518290,4.645900,1.0,5.0,7.0,10.0,25.0
lt_gr_id,502493.0,6.890339,1.525189,1.0,6.0,8.0,8.0,8.0
nps_score,502493.0,7.508562,3.020378,1.0,5.0,9.0,10.0,10.0


Присутствуют пропуски в столбцах `age` и `gender_segment`. Первое для нас не критично, так как сегментацию по возрасту мы будем смотреть через поле `age_gr_id` в таблице `age_segment`. 

В процессе формирования SQL запроса необходимо учесть, что в поле `gender_segment` помимо очевидных "мужчина" и "женщина" есть значение "нет данных". Таких значений менее 1% от всей выборки. Для анализа эти данные вряд ли пригодяться, поэтому принимаем решение удалить их. Проверим, нет ли других значений в этом столбце.

In [5]:
user.gender_segment.value_counts()

1.0    272442
0.0    228750
Name: gender_segment, dtype: int64

Все в порядке. А еще мы знаем, что женщин и мужчин примерно поровну с небольшим преимуществом у первых.

Также с помощью `.describe()` в поле `lt_day` обнаружены отрицательные значения. Техническим заданием к исследованию запрещается использовать python для преобразования данных, но для принятия решения, что делать с этими данными, необходимо понять сколько их. Посмотрим на столбец.

In [6]:
user.query('lt_day < 0')

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
7666,AEAC2R,-8,NaN,NaN,ANDROID,SMARTPHONE,50,8,8,8,1
53997,CS0HF8,-2,NaN,NaN,ANDROID,SMARTPHONE,15,8,9,8,10
71577,DORPT2,-4,NaN,NaN,ANDROID,SMARTPHONE,28,8,9,8,4
100152,F5O3CG,-21,NaN,NaN,ANDROID,SMARTPHONE,60,8,12,8,7
228087,LSE939,-13,NaN,NaN,ANDROID,SMARTPHONE,22,8,8,8,1
325212,QTT7IR,-13,NaN,NaN,IOS,SMARTPHONE,28,8,7,8,7
347784,RZUS6H,-6,NaN,NaN,ANDROID,SMARTPHONE,23,8,11,8,3
385897,TYWQW4,-12,NaN,NaN,IOS,SMARTPHONE,33,8,5,8,5
401226,URKAFI,-11,NaN,NaN,ANDROID,SMARTPHONE,30,8,7,8,8
407160,V2GNK2,-13,NaN,NaN,IOS,SMARTPHONE,38,8,8,8,5


Всего 11 значений. В процессе формирования SQL запроса избавимся от них.

### Таблица location

In [7]:
location = pd.read_sql('SELECT * FROM location;', engine)
location.head(10)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия
5,6,Брянск,Россия
6,7,Владивосток,Россия
7,8,Владимир,Россия
8,9,Волгоград,Россия
9,10,Волжский,Россия


Название город и стран, все понятно. Посмотрим, в каких странах проводили опрос

In [8]:
location.country.value_counts()

Россия    62
Name: country, dtype: int64

Теперь мы знаем, что опрос проводили только в России в 62 городах. Посмотрим на таблицу с возрастными категориями.

### Таблица age_segment

In [9]:
age_segment = pd.read_sql('SELECT * FROM age_segment;', engine)
age_segment.head(10)

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


Логика понятна, `bucket_min` это от какого возраста начинается категория, а `bucket_max` на каком возрасте она заканчиается. Интересно куда делись люди от 64 до 66 лет, но задать этот вопрос нам некому.

Названия категорий нечитабельны, будем переписывать. Посмотрим на таблицу с сегментацией по потребляемому трафику

### Таблица traffic_segment

In [10]:
traffic_segment = pd.read_sql('SELECT * FROM traffic_segment;', engine)
traffic_segment

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5
5,6,5.00,10.00,05 5-10
6,7,10.00,15.00,06 10-15
7,8,15.00,20.00,07 15-20
8,9,20.00,25.00,08 20-25
9,10,25.00,30.00,09 25-30


Логика та-же, что и в предыдущей таблице и такие же проблемы с названиями. Посмотрим на последнюю таблицу по лайвтайму.

### Таблица lifetime_segment

In [11]:
lifetime_segment = pd.read_sql('SELECT * FROM lifetime_segment;', engine)
lifetime_segment

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


Все ясно, приступим к формированию SQL запроса.

### Выводы

- Таблица user является ключевой, ее будем объединять с остальными по `_id` в других таблицах
- В столбце `age_segment` таблицы `user` только три значения - `1`,  `0` и `NaN`. От последнего принято решение избавиться.
- В столбце `lt_day` 11 отрицательных значений. Принято решение избавиться от них во время формирования SQL запроса.
- Из таблицы `location` узнали, что опрос проходил только в России в 62 городах
- Таблицы `age_segment`, `traffic_segment` и `lifetime_segment` имеют схожую структуру, во всех них требуется переименовать название категорий

## Формирование SQL запроса

По условиям технического задания к исследованию, создание витрины должны быть осуществлено только с ипользованием SQL запроса. В витрине должны быть следующие поля: 


Название |Описание |
:--------|:--------|
user_id|Идентификатор клиента|
lt_day|Количество дней «жизни» клиента|
is_new|Поле хранит информацию о том, является ли клиент новым|
age|Возраст|
gender_segment|Пол (для удобства работы с полем преобразуйте значения в текстовый вид)|
os_name|Тип операционной системы|
cpe_type_name|Тип устройства|
country|Страна проживания|
city|Город проживания|
age_segment|Возрастной сегмент|
traffic_segment|Сегмент по объёму потребляемого трафика|
lifetime_segment|Сегмент по количеству дней «жизни»|
nps_score|Оценка клиента в NPS-опросе|
nps_group|Поле хранит информацию о том, к какой группе относится оценка клиента в опросе|

Также добавим поле `total_nps` для упрощения расчета NPS в Tableau

In [12]:
query = """
WITH
new_users AS (SELECT user_id,
                     CASE
                         WHEN lt_day <= 365 THEN 'Новый клиент'
                         ELSE 'Старый клиент'
                     END AS is_new
              FROM user),
              
gender AS (SELECT user_id,
                  CASE
                      WHEN gender_segment = 1 THEN 'Женщина'
                      WHEN gender_segment = 0 THEN 'Мужчина'
                      ELSE 'Нет данных'
                  END AS gender_segment_norm
           FROM user),
           
group_nps AS (SELECT user_id,
                     CASE
                         WHEN nps_score BETWEEN 0 AND 6 THEN 'Критики'
                         WHEN nps_score BETWEEN 7 AND 8 THEN 'Нейтралы'
                         WHEN nps_score BETWEEN 9 AND 10 THEN 'Сторонники'
                     END AS nps_group
              FROM user),
              
total_nps AS (SELECT user_id,
                     CASE
                         WHEN nps_score BETWEEN 0 AND 6 THEN -1
                         WHEN nps_score BETWEEN 7 AND 8 THEN 0
                         WHEN nps_score BETWEEN 9 AND 10 THEN 1
                     END AS nps_total
              FROM user),
              
lifitime_segment_normal AS (SELECT lt_gr_id,
                                   CASE
                                       WHEN lt_gr_id = 1 THEN '1 месяц'
                                       WHEN lt_gr_id = 2 THEN '2 месяца'
                                       WHEN lt_gr_id = 3 THEN '3 месяца'
                                       WHEN lt_gr_id = 4 THEN '4-6 месяцев'
                                       WHEN lt_gr_id = 5 THEN '7-12 месяцев'
                                       WHEN lt_gr_id = 6 THEN '13-24 месяцев'
                                       WHEN lt_gr_id = 7 THEN '25-36 месяцев'
                                       WHEN lt_gr_id = 8 THEN 'более 36 месяцев'
                                    END AS title_normal
                            FROM lifetime_segment),
                            
age_segment_normal AS (SELECT age_gr_id,
                                   CASE
                                       WHEN age_gr_id = 1 THEN 'до 16 лет'
                                       WHEN age_gr_id = 2 THEN 'от 16 до 24 лет'
                                       WHEN age_gr_id = 3 THEN 'от 25 до 34 лет'
                                       WHEN age_gr_id = 4 THEN 'от 35 до 44 лет'
                                       WHEN age_gr_id = 5 THEN 'от 45 до 54 лет'
                                       WHEN age_gr_id = 6 THEN 'от 55 до 64 лет'
                                       WHEN age_gr_id = 7 THEN 'более 66 лет'
                                       WHEN age_gr_id = 8 THEN 'нет данных'
                                    END AS title_normal
                            FROM age_segment),
                            
traffic_segment_normal AS (SELECT tr_gr_id,
                                   CASE
                                       WHEN tr_gr_id = 1 THEN '0'
                                       WHEN tr_gr_id = 2 THEN 'от 0 до 0.01'
                                       WHEN tr_gr_id = 3 THEN 'от 0.01 до 0.1'
                                       WHEN tr_gr_id = 4 THEN 'от 0.1 до 1'
                                       WHEN tr_gr_id = 5 THEN 'от 1 до 5'
                                       WHEN tr_gr_id = 6 THEN 'от 5 до 10'
                                       WHEN tr_gr_id = 7 THEN 'от 10 до 15'
                                       WHEN tr_gr_id = 8 THEN 'от 15 до 20'
                                       WHEN tr_gr_id = 9 THEN 'от 20 до 25'
                                       WHEN tr_gr_id = 10 THEN 'от 25 до 30'
                                       WHEN tr_gr_id = 11 THEN 'от 30 до 35'
                                       WHEN tr_gr_id = 12 THEN 'от 35 до 40'
                                       WHEN tr_gr_id = 13 THEN 'от 40 до 45'
                                       WHEN tr_gr_id = 14 THEN 'от 45 до 50'
                                       WHEN tr_gr_id = 15 THEN 'от 50 до 55'
                                       WHEN tr_gr_id = 16 THEN 'от 55 до 60'
                                       WHEN tr_gr_id = 17 THEN 'от 60 до 65'
                                       WHEN tr_gr_id = 18 THEN 'от 65 до 70'
                                       WHEN tr_gr_id = 19 THEN 'от 70 до 75'
                                       WHEN tr_gr_id = 20 THEN 'от 75 до 80'
                                       WHEN tr_gr_id = 21 THEN 'от 80 до 85'
                                       WHEN tr_gr_id = 22 THEN 'от 85 до 90'
                                       WHEN tr_gr_id = 23 THEN 'от 90 до 95'
                                       WHEN tr_gr_id = 24 THEN 'от 95 до 100'
                                       WHEN tr_gr_id = 25 THEN '100+'
                                    END AS title_normal
                            FROM traffic_segment)
              
              
SELECT u.user_id,
       u.lt_day,
       nu.is_new,
       u.age,
       g.gender_segment_norm AS gender_segment,       
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       ags.title_normal AS age_segment,
       ts.title_normal AS traffic_segment,
       ls.title_normal AS lifetime_segment,
       u.nps_score,
       gn.nps_group,
       tn.nps_total
       
FROM user AS u
JOIN new_users AS nu ON u.user_id = nu.user_id
JOIN gender AS g ON u.user_id = g.user_id
JOIN location AS l ON u.location_id = l.location_id
JOIN age_segment_normal AS ags ON u.age_gr_id = ags.age_gr_id
JOIN traffic_segment_normal AS ts ON u.tr_gr_id = ts.tr_gr_id
JOIN lifitime_segment_normal AS ls ON u.lt_gr_id = ls.lt_gr_id
JOIN group_nps AS gn ON u.user_id = gn.user_id
JOIN total_nps AS tn ON u.user_id = tn.user_id
WHERE u.lt_day >= 0 AND
      u.gender_segment IS NOT NULL;

"""

Посмотрим на результат запроса

In [13]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group,nps_total
0,A001A2,2320,Старый клиент,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,от 45 до 54 лет,от 1 до 5,более 36 месяцев,10,Сторонники,1
1,A001WF,2344,Старый клиент,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,от 45 до 54 лет,от 1 до 5,более 36 месяцев,10,Сторонники,1
2,A003Q7,467,Старый клиент,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,от 55 до 64 лет,от 20 до 25,13-24 месяцев,10,Сторонники,1
3,A004TB,4190,Старый клиент,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,от 35 до 44 лет,от 0.1 до 1,более 36 месяцев,10,Сторонники,1
4,A004XT,1163,Старый клиент,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,от 16 до 24 лет,от 5 до 10,более 36 месяцев,10,Сторонники,1
5,A005O0,5501,Старый клиент,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Омск,от 35 до 44 лет,от 5 до 10,более 36 месяцев,6,Критики,-1
6,A0061R,1236,Старый клиент,45.0,Мужчина,ANDROID,SMARTPHONE,Россия,Уфа,от 45 до 54 лет,от 10 до 15,более 36 месяцев,10,Сторонники,1
7,A009KS,313,Новый клиент,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,от 35 до 44 лет,от 45 до 50,7-12 месяцев,10,Сторонники,1
8,A00AES,3238,Старый клиент,36.0,Женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,от 35 до 44 лет,от 1 до 5,более 36 месяцев,10,Сторонники,1
9,A00F70,4479,Старый клиент,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Волгоград,от 45 до 54 лет,от 15 до 20,более 36 месяцев,9,Сторонники,1


Проверим, все ли сработало верно

In [14]:
df.info()
df.describe().T
display(df.gender_segment.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501192 entries, 0 to 501191
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           501192 non-null  object 
 1   lt_day            501192 non-null  int64  
 2   is_new            501192 non-null  object 
 3   age               501152 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           501192 non-null  object 
 6   cpe_type_name     501192 non-null  object 
 7   country           501192 non-null  object 
 8   city              501192 non-null  object 
 9   age_segment       501192 non-null  object 
 10  traffic_segment   501192 non-null  object 
 11  lifetime_segment  501192 non-null  object 
 12  nps_score         501192 non-null  int64  
 13  nps_group         501192 non-null  object 
 14  nps_total         501192 non-null  int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 57.4+ MB


Женщина    272442
Мужчина    228750
Name: gender_segment, dtype: int64

Все сработало как и ожидалось. Выгрузим файл в `.csv` и приступим к построению дэшборда и анализу

In [15]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Дэшборд Tablaeu

https://public.tableau.com/app/profile/andrei5358/viz/yandex_telec_nps/sheet10?publish=yes

## Презентация

https://drive.google.com/file/d/1lScf2JI9Ffdt1Ed-Dm0QEkcX3WgFBr_1/view?usp=sharing